# Accurate Integer Addition in Transformers

This CoLab defines, trains and analyses a Transformer model that performs integer addition e.g. 33357+82243=115600. Each digit is a separate token. For 5 digit addition, the model is given 12 "question" (input) tokens, and must then predict the corresponding 6 "answer" (output) tokens.


ProblemShape.svg

The CoLab follows on from the [Understanding Addition in Transformers](https://colab.research.google.com/drive/1p71dC3LCPJIfFKqr2rhmBZBOyyMpovyn) which explains integer addition and documents a rare high-loss use case called "Use Sum 9 Cascade". This CoLab seeks to eliminate this high-loss use case.

## Tips for using the Colab
 * You can run and alter the code in this CoLab notebook yourself in Google CoLab ( https://colab.research.google.com/ ).
 * To run the notebook, in Google CoLab, **you will need to** go to Runtime > Change Runtime Type and select GPU as the hardware accelerator.
 * Some graphs are interactive!
 * Use the table of contents pane in the sidebar to navigate.
 * Collapse irrelevant sections with the dropdown arrows.
 * Search the page using the search in the sidebar, not CTRL+F.

# Part 1: Setup
Imports standard libraries. Don't bother reading. Skip


In [ ]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEVELOPMENT_MODE = True
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install kaleido
    #%pip install git+https://github.com/neelnanda-io/TransformerLens.git@new-demo
    %pip install transformer_lens
    %pip install git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python
    #%pip install circuitsvis
    %pip install jaxtyping
    %pip install einops
    %pip install fancy_einsum
    %pip install torchtyping
    %pip install transformers
    %pip install datasets

    %pip install prettytable

except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook


In [ ]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import kaleido
import plotly.io as pio

if IN_COLAB or not DEVELOPMENT_MODE:
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
pio.templates['plotly'].layout.xaxis.title.font.size = 20
pio.templates['plotly'].layout.yaxis.title.font.size = 20
pio.templates['plotly'].layout.title.font.size = 30

In [ ]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import random
from pathlib import Path
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from torchtyping import TensorType as TT
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML
from IPython.display import display

import circuitsvis as cv

from prettytable import PrettyTable

In [ ]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

In [ ]:
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

# Part 2: Configuration
This section defines the token embedding / unembedding and creates the model.

The model has been successfully trained to do 2, 5, 10, 15 digit integer addition. The default is n_digits = 5.

The model has been successfully trained with 2, 3 or 4 attention heads. The  default is n_heads = 3. More heads do not increase accuracy

The model has been successfully trained with 1 or 2 layers. The default is n_layers = 2. Two layers increases accuracy in Cascading UseSum9 cases over 1 layer.

long_equals changes the question format "12345+11111=023465" to "12345+11111equals023465". This does not increase accuracy.

Setting more_ms9_cases to true (to increase the percentage of UseSum9 cases in the training data) speeds up training, but doesnt increase trained accuracy.  



In [ ]:
more_ms9_cases = True # When doing addition, increase frequency of Make Sum 9 cases in training data
long_equals = False # If true, in question formats use "equals" instead of "="


In [ ]:
# Tokens used in vocab. (Token indexes 0 to 9 represent digits 0 to 9)
PLUS_INDEX = 10
MINUS_INDEX = 11
EQUALS_INDEX = 12
E_INDEX = 13
Q_INDEX = 14
U_INDEX = 14
A_INDEX = 16
L_INDEX = 17
S_INDEX = 18

#@markdown Model
n_layers = 2 #@param
d_vocab = (S_INDEX+1 if long_equals == True else EQUALS_INDEX+1)
n_heads = 3 #@param
d_model = ( 512 // n_heads ) * n_heads    # About 512, and divisible by n_heads
d_head = d_model // n_heads               # About 170 when n_heads == 3
d_mlp = 4 * d_model
seed = 129000 #@param

#@markdown Data
n_digits = 5 #@param
n_ctx = 3 * n_digits + (8 if long_equals == True else 3)
act_fn = 'relu'
batch_size = 64 #@param

#@markdown Optimizer
lr = 0.00008 #@param
weight_decay = 0.1 #@param
n_training_steps = 20000


In [ ]:
# Train or load model? Training saves the model weights in a temporary CoLab file
train_model = True #@param

# The name of the temporary CoLab file to save model to
pth_location = "model.pth"

# Save graphs to files as PDF and HTML
save_graph_to_file = False

# The format to output prettytable in
# Supports text|html|json|csv|latex
table_out_format = "text"

In [ ]:
# Embedding / Unembedding

def tokens_to_string(tokens):
    tokens = utils.to_numpy(tokens)
    x = "".join([str(i) for i in tokens[:n_digits]])
    y = "".join([str(i) for i in tokens[n_digits+1:n_digits*2+1]])
    z = "".join([str(i) for i in tokens[n_ctx-n_digits-1:]])
    equals = "equals" if long_equals == True else "="
    operator = "+"
    return f"{x}{operator}{y}{equals}{z}"

def string_to_tokens(string, batch=False):
    lookup = {str(i):i for i in range(10)}
    lookup['+']=PLUS_INDEX
    lookup['-']=MINUS_INDEX
    lookup['=']=EQUALS_INDEX

    if long_equals == True:
      lookup['e']=E_INDEX
      lookup['q']=Q_INDEX
      lookup['u']=U_INDEX
      lookup['a']=A_INDEX
      lookup['l']=L_INDEX
      lookup['s']=S_INDEX

    tokens = [lookup[i] for i in string if i not in '\n ']
    if batch:
        return torch.tensor(tokens)[None, :]
    else:
        return torch.tensor(tokens)

In [ ]:
# Transformer creation

# Structure is documented at https://neelnanda-io.github.io/TransformerLens/transformer_lens.html#transformer_lens.HookedTransformerConfig.HookedTransformerConfig
cfg = HookedTransformerConfig(
    n_layers = n_layers,
    n_heads = n_heads,
    d_model = d_model,
    d_head = d_head,
    d_mlp = d_mlp,
    act_fn = act_fn,
    normalization_type = 'LN',
    d_vocab=d_vocab,
    d_vocab_out=d_vocab,
    n_ctx=n_ctx,
    init_weights = True,
    device="cuda",
    seed = seed,
)

model = HookedTransformer(cfg)

optimizer = optim.AdamW(model.parameters(),
                        lr=lr,
                        weight_decay=weight_decay,
                        betas=(0.9, 0.98))

scheduler = optim.lr_scheduler.LambdaLR(optimizer, lambda step: min(step/10, 1))

# Part 3: Data Generator. Addition sub-task categorisation
This section defines the loss function and the training/tesing data generator.

It also defines functions to categorise the training data by the addition sub-task defined in the paper. The addition sub tasks are abbreviated as:
- BA is Base Add. Calculates the sum of two digits Dn and Dn' modulo 10, ignoring any carry over from previous columns.
- MC1 is Make Carry 1. Evaluates to true if adding digits Dn and Dn' results in a carry over of 1 to the next column.
- MS9 is Make Sum 9. Evaluates to true if adding digits Dn and Dn' gives exactly 9.
- UC1 is Use Carry 1. Takes the previous column's carry output and adds it to the sum of the current digit pair.
- US9 is Use Sum 9. Propagates (aka cascades) a carry over of 1 to the next column if the current column sums to 9 and the previous column generated a carry over. US9 is the most complex task as it spans three digits. For some rare questions (e.g. 00555 + 00445 = 01000) US9 applies to up to four sequential digits, causing a chain effect, with the MC1 cascading through multiple digits.

In [ ]:
# Loss functions

# Calculate the per-token probability by comparing a batch of prediction "logits" to answer "tokens"
def logits_to_tokens_loss(logits, tokens):

  # The last "n_digit+1" tokens are the addition answer probabilities
  ans_logits = logits[:, -(n_digits+2):-1]

  # Convert raw score (logits) vector into a probability distribution.
  # Emphasize the largest scores and suppress the smaller ones, to make them more distinguishable.
  ans_probs = F.log_softmax(ans_logits.to(torch.float64), dim=-1)

  max_indices = torch.argmax(ans_probs, dim=-1)

  # The last "n_digit+1" tokens are the model’s answer.
  ans_tokens = tokens[:, -(n_digits+1):]

  # Extract values from the ans_probs tensor, based on indices from the ans_tokens tensor
  ans_loss = torch.gather(ans_probs, -1, ans_tokens[:, :, None])[..., 0]

  return ans_loss, max_indices

# Calculate loss as negative of average per-token mean probability
def loss_fn(ans_loss):
  return -ans_loss.mean(0)

In [ ]:
# Define "iterator" data generator function. Invoked using next().
# "Addition" batch entries are formated XXXXX+YYYYY=ZZZZZZ e.g. 55003+80002=135005
# "Subtraction" batch entries are formated XXXXX-YYYYY=ZZZZZZ e.g. 55003-80002=-24999, 80002-55003=024999
# Note that answer has one more digit than the question
# Returns characteristics of each batch entry to aid later analysis
def data_generator(batch_size, n_digits, seed):
    torch.manual_seed(seed)
    while True:
        #generate a batch of questions (answers calculated below)
        batch = torch.zeros((batch_size, n_ctx)).to(torch.int64)
        x = torch.randint(0, 10, (batch_size, n_digits))
        y = torch.randint(0, 10, (batch_size, n_digits))

        if more_ms9_cases == True:
          # The UseSum9 task is compound and rare and so hard to learn.
          # For some of batches, we increase the MakeSum9 case frequency
          # UseSum9 also relies on MakeCarry1 (50%) from previous column.
          # So UseSum9 frequency is increased by 60% * 40% * 50% = 12%
          if random.randint(1, 5) < 3: # 60%
            # Flatten x and y to 1D tensors
            x_flat = x.view(-1)
            y_flat = y.view(-1)

            num_elements_to_modify = int(0.40 * x.numel()) # 40%
            indices_to_modify = torch.randperm(x_flat.numel())[:num_elements_to_modify]
            if random.randint(1, 2) == 1:
              x_flat[indices_to_modify] = 9 - y_flat[indices_to_modify]
            else:
              y_flat[indices_to_modify] = 9 - x_flat[indices_to_modify]

            # Reshape x and y back to its original shape
            x = x_flat.view(x.shape)
            y = y_flat.view(x.shape)

        batch[:, :n_digits] = x
        batch[:, n_digits] = PLUS_INDEX
        batch[:, 1+n_digits:1+n_digits*2] = y
        if long_equals == True:
          batch[:, 1+n_digits*2] = E_INDEX
          batch[:, 2+n_digits*2] = Q_INDEX
          batch[:, 3+n_digits*2] = U_INDEX
          batch[:, 4+n_digits*2] = A_INDEX
          batch[:, 5+n_digits*2] = L_INDEX
          batch[:, 6+n_digits*2] = S_INDEX
        else:
          batch[:, 1+n_digits*2] = EQUALS_INDEX

        # These attributes are used for testing addition
        base_adds = torch.zeros((batch_size,n_digits)).to(torch.int64)
        make_carry1s = torch.zeros((batch_size,n_digits)).to(torch.int64)
        sum9s = torch.zeros((batch_size,n_digits)).to(torch.int64)
        use_carry1s = torch.zeros((batch_size,n_digits)).to(torch.int64)
        use_sum9s = torch.zeros((batch_size,n_digits)).to(torch.int64)

        # generate the addition question answers & other info for testing
        for i in range(n_digits):
            # the column in the test attributes being updated
            test_col = n_digits-1-i

            base_add = batch[:, n_digits-1-i] + batch[:, 2*n_digits-i]
            base_adds[:, test_col] = base_add % 10

            sum9 = (base_add == 9)
            sum9s[:, test_col] = sum9

            if i>0:
              use_carry1s[:, test_col] = make_carry1s[:, test_col+1]
            use_carry = use_carry1s[:, test_col]

            use_sum9s[:, test_col] = sum9 & use_carry;

            digit_sum = base_add + use_carry1s[:, test_col]

            make_carry = (digit_sum >= 10)
            make_carry1s[:, test_col] = make_carry

            batch[:, -1-i] = (digit_sum % 10)

        # Final (possible) carry to highest digit of the sum
        batch[:, -1-n_digits] = make_carry1s[:, 0]

        yield batch.cuda(), base_adds.cuda(), make_carry1s.cuda(), sum9s.cuda(), use_carry1s.cuda(), use_sum9s.cuda()

In [ ]:
ds = data_generator(batch_size, n_digits, seed)

tokens, base_adds, make_carry1s, sum9s, use_carry1s, use_sum9s = next(ds)

print(tokens[0])

In [ ]:
# Data generator unit test (optional)
# This unit test checks that the above data_generator function is sensible
def unit_test_data_generator(train_tokens, train_use_carry1s, train_make_carry1s):
  test_token = train_tokens[0]
  test_use_carry = train_use_carry1s[0]
  test_make_carry = train_make_carry1s[0]

  if n_digits == 5:
    digits = test_token.cpu().numpy()
    use = test_use_carry.cpu().numpy()
    force = test_make_carry.cpu().numpy()

    num1 = digits[0]*10000 + digits[1]*1000 + digits[2]*100 + digits[3]*10 + digits[4];
    num2 = digits[6]*10000 + digits[7]*1000 + digits[8]*100 + digits[9]*10 + digits[10];
    if long_equals == True:
      sum = digits[17]*100000 + digits[18]*10000 + digits[19]*1000 + digits[20]*100 + digits[21]*10 + digits[22];
    else:
      sum = digits[12]*100000 + digits[13]*10000 + digits[14]*1000 + digits[15]*100 + digits[16]*10 + digits[17];

    assert num1 + num2 == sum, "Unit test failed: Data generator: Bad sum"
    assert (digits[4]+digits[10]+use[4]>=10) == force[4], "Unit test failed: Data generator: Bad carry 0"
    assert (digits[3]+digits[9]+use[3]>=10) == force[3], "Unit test failed: Data generator: Bad carry 1"
    assert (digits[2]+digits[8]+use[2]>=10) == force[2], "Unit test failed: Data generator: Bad carry 2"
    assert (digits[1]+digits[7]+use[1]>=10) == force[1], "Unit test failed: Data generator: Bad carry 3"
    assert (digits[0]+digits[6]+use[0]>=10) == force[0], "Unit test failed: Data generator: Bad carry 4"

In [ ]:
# Base-Add-only loss
# Identify the subset of (simple) tokens that only require BA (not UC1 or US9) to get the correct answer
# Array index 0 is the 'Units' digit. Array index 3 is the 'Thousands' digit.
ba_alldigits_loss = []
ba_alldigits_oneloss = 0

ba_perdigit_loss = []
ba_perdigit_cases = 0
ba_total_cases = 0


# Base Add AllDigits
# Identity the tokens in the batch where UC1 is false for all columns simultaneously, so only BA is required on all digits
def calculate_ba_oneloss(tokens, per_token_losses, base_adds, use_carry1s):
  global ba_alldigits_oneloss

  answer = 0
  any_use_carry1s = torch.any(use_carry1s.bool(), dim=1)
  no_use_carry1s = ~ any_use_carry1s
  num_cases = utils.to_numpy(torch.sum(no_use_carry1s))
  if num_cases > 0 :
    filtered_loss = per_token_losses[:, -n_digits:] * no_use_carry1s[:, None]
    sum_loss = torch.sum(filtered_loss)
    answer = - utils.to_numpy(sum_loss) / num_cases
    answer = answer / n_digits  # Approximately align the scale of ba_alldigits_loss to ba_perdigit_loss
  ba_alldigits_oneloss = answer


def calculate_ba_loss(tokens, per_token_losses, base_adds, use_carry1s):
  global ba_perdigit_cases
  global ba_total_cases

  # Base Add All Digits
  # Identity the tokens in the batch where UC1 is false for all columns simultaneously, so only BA is required on all digits
  calculate_ba_oneloss(tokens, per_token_losses, base_adds, use_carry1s)
  ba_alldigits_loss.append(ba_alldigits_oneloss)


  # Base Add Per Digit
  # For each token in the batch, identity the digit columns (e.g. 3) where use_carry is false, so only BA is required on that digit
  ba_perdigit_cases = 0;
  for digit_num in range(n_digits):
    answer = 0
    no_use_carry = 1 - use_carry1s[:, -1-digit_num]
    num_cases = utils.to_numpy(torch.sum(no_use_carry))
    ba_perdigit_cases += num_cases
    ba_total_cases += num_cases
    if num_cases > 0 :
      filtered_loss = per_token_losses[:, -1-digit_num] * no_use_carry
      sum_loss = torch.sum(filtered_loss)
      answer = - utils.to_numpy(sum_loss) / num_cases
    if len(ba_perdigit_loss)<=digit_num:
      ba_perdigit_loss.append([])
    if (num_cases == 0) & (len(ba_perdigit_loss[digit_num]) > 0) :
      answer = ba_perdigit_loss[digit_num][-1] # Use the previous step's loss. Improves graph
    ba_perdigit_loss[digit_num].append(answer)

In [ ]:
# Use Carry 1 loss
# Identify the subset of tokens that require UC1 (but not US9) to get the correct answer
# Array index 0 is the 'Units' digit. Array index 3 is the 'Thousands' digit.
uc1_anydigits_loss = []
uc1_anydigits_oneloss = 0

uc1_perdigit_loss = []
uc1_perdigit_cases = 0
uc1_total_cases = 0


# UC1 AnyDigits (exclude Sum9)
# Identity the tokens in the batch where UC1 is used at least once over the columns & Sum9 is never used
def calculate_uc1_loss_any(tokens, per_token_losses, use_carry1s, sum9s):
  global uc1_anydigits_oneloss

  num_use_carry1s = torch.sum(use_carry1s, dim=1)
  any_use_carry1s = torch.where( num_use_carry1s != 0, 1, 0 ) # At least one digit uses UC1
  num_sum9s = torch.sum(use_sum9s, dim=1)
  no_sum9s = torch.where( num_sum9s == 0, 1, 0 ) # No digits have Sum9 true
  filtered_cases = any_use_carry1s & no_sum9s
  num_cases = utils.to_numpy(torch.sum(filtered_cases))
  filtered_indices = torch.nonzero(filtered_cases).squeeze()
  filtered_token_losses = per_token_losses[filtered_indices]
  answer = - filtered_token_losses.mean()
  uc1_anydigits_oneloss = utils.to_numpy(answer)


def calculate_uc1_loss(tokens, per_token_losses, use_carry1s, sum9s):
  global uc1_perdigit_cases
  global uc1_total_cases

  # UC1 AnyDigits (exclude Sum9)
  # Identity the tokens in the batch where UC1 is used at least once over the columns & Sum9 is never used
  calculate_uc1_loss_any(tokens, per_token_losses, use_carry1s, sum9s)
  uc1_anydigits_loss.append(uc1_anydigits_oneloss)

  # UC1 PerDigit (exclude Sum9)
  # For each token in the batch, identity the digit columns (e.g. 3) where UC1 is used on the columns & Sum9 is not true
  uc1_perdigit_cases = 0
  for digit_num in range(n_digits):
    answer = 0
    use_carry = use_carry1s[:, -1-digit_num]
    no_sum9 = 1 - sum9s[:, -1-digit_num]
    filtered_cases = use_carry & no_sum9
    num_cases = utils.to_numpy(torch.sum(filtered_cases))
    uc1_perdigit_cases += num_cases
    uc1_total_cases += num_cases
    if num_cases > 0 :
      filtered_loss = per_token_losses[:, -1-digit_num] * filtered_cases
      sum_loss = torch.sum(filtered_loss)
      answer = - utils.to_numpy(sum_loss) / num_cases
    if len(uc1_perdigit_loss)<=digit_num:
      uc1_perdigit_loss.append([])
    if (num_cases==0) & (len(uc1_perdigit_loss[digit_num]) > 0) :
      answer = uc1_perdigit_loss[digit_num][-1] # Use the previous step's loss. Improves graph
    uc1_perdigit_loss[digit_num].append(answer)

In [ ]:
# Use Sum 9 loss
# Identify the subset of tokens that require US9 (being Sum9 and Carry1 from prev column) to get the correct answer
# Array index 0 is the 'Units' digit. Array index 3 is the 'Thousands' digit.
us9_anydigits_loss = []
us9_anydigits_oneloss = 0

us9_perdigit_loss = []
us9_perdigit_cases = 0
us9_total_cases = 0


# US9 OneDigit
# Identity the tokens in the batch where US9 is used at least once over the columns
def calculate_us9_oneloss(tokens, per_token_losses, use_sum9s):
  global us9_anydigits_oneloss

  num_use_sum9s = torch.sum(use_sum9s, dim=1)
  filtered_num_use_sum9s = torch.where( num_use_sum9s != 0, 1, 0 ) # At least OneDigit uses US9
  num_cases = utils.to_numpy(torch.sum(filtered_num_use_sum9s))
  filtered_indices = torch.nonzero(filtered_num_use_sum9s).squeeze()
  filtered_token_losses = per_token_losses[filtered_indices]
  answer = - filtered_token_losses.mean()
  us9_anydigits_oneloss = utils.to_numpy(answer);


def calculate_us9_loss(tokens, per_token_losses, use_sum9s):
  global us9_perdigit_cases
  global us9_total_cases

  # US9 OneDigit
  # Identity the tokens in the batch where US9 is used at least once over the columns
  calculate_us9_oneloss(tokens, per_token_losses, use_sum9s)
  us9_anydigits_loss.append(us9_anydigits_oneloss)

  # For each token in the batch, identity the digit columns (e.g. 3) where US9 is used
  us9_perdigit_cases = 0
  for digit_num in range(n_digits):
    answer = 0
    use_carry = use_carry1s[:, -1-digit_num]
    use_sum9 = sum9s[:, -1-digit_num]
    filtered_cases = use_carry & use_sum9
    num_cases = utils.to_numpy(torch.sum(filtered_cases))
    us9_perdigit_cases += num_cases
    us9_total_cases += num_cases
    if num_cases > 0 :
      filtered_loss = per_token_losses[:, -1-digit_num] * filtered_cases
      sum_loss = torch.sum(filtered_loss)
      answer = - utils.to_numpy(sum_loss) / num_cases
    if len(us9_perdigit_loss)<=digit_num:
      us9_perdigit_loss.append([])
    if (num_cases==0) & (len(us9_perdigit_loss[digit_num]) > 0) :
      answer = us9_perdigit_loss[digit_num][-1] # Use the previous step's loss. Improves graph
    us9_perdigit_loss[digit_num].append(answer)


In [ ]:
# Check that us9_perdigit_loss, uc1_perdigit_loss and ba_perdigit_loss do NOT overlap
# This ensures the graphs of each are non-overlapping
def unit_test_nonoverlapping():
  global ba_perdigit_cases
  global ba_total_cases
  global uc1_perdigit_cases
  global uc1_total_cases
  global us9_perdigit_cases
  global us9_total_cases

  perdigit_numcases = us9_perdigit_cases + uc1_perdigit_cases + ba_perdigit_cases
  assert (perdigit_numcases == batch_size * n_digits), "Cases overlap: " + str(perdigit_numcases) + " != " + str(batch_size*n_digits)

# Part 4: Train model with Infinite Data
Train model for n_training_steps, storing train_losses per epoch.

Each training step (of n_training_steps) new training data (a batch of batch_size tokens) is generated and the model is trained and loss calculated on it. No separate "testing" data is needed, as the training data is unique each step. Memorisation of past training data by the model (if any) is minimally beneficial. For 5 digit addition there are 10 billion possible questions, and model training is on ~2 million questions.

In [ ]:
def print_config():
  print("n_digits=", n_digits, "n_heads=", n_heads, "n_layers=", n_layers, "n_ctx=", n_ctx, "seed=", seed, "long_equals=", long_equals, "n_training_steps=", n_training_steps)

In [ ]:
# Initialise the data generator
ds = data_generator(batch_size, n_digits, seed)

if train_model:
  # Train the model
  train_losses_list = []
  per_token_train_losses_list = []

  for epoch in tqdm.tqdm(range(n_training_steps)):

      tokens, base_adds, make_carry1s, sum9s, use_carry1s, use_sum9s = next(ds)
      logits = model(tokens)

      per_token_train_losses_raw, _ = logits_to_tokens_loss(logits, tokens)
      per_token_train_losses = loss_fn(per_token_train_losses_raw)
      per_token_train_losses_list.append(utils.to_numpy(per_token_train_losses))

      train_loss = per_token_train_losses.mean()
      train_loss.backward()
      train_losses_list.append(train_loss.item())

      calculate_ba_loss(tokens, per_token_train_losses_raw, base_adds, use_carry1s)
      calculate_uc1_loss(tokens, per_token_train_losses_raw, use_carry1s, sum9s)
      calculate_us9_loss(tokens, per_token_train_losses_raw, use_sum9s)

      optimizer.step()
      scheduler.step()
      optimizer.zero_grad()

      if epoch % 100 == 0:
          print(epoch, train_loss.item())
          unit_test_data_generator(tokens, use_carry1s, make_carry1s)
          unit_test_nonoverlapping()


In [ ]:
# Even at the end of training, the loss can wobble between epochs, perhaps based on #of rare edge cases in the training data.
# Use the average of last 5 training losses as the "final accuracy"
print_config()
print( "Final training loss", round((train_losses_list[-5]+train_losses_list[-4]+train_losses_list[-3]+train_losses_list[-2]+train_losses_list[-1])/5,6))

In [ ]:
if train_model:
  # Save the model to file
  torch.save(model.state_dict(), pth_location)
else:
  # Load the model from file
  model.load_state_dict(torch.load(pth_location))
  model.eval()

# Part 5: Training Loss Analysis - High Level Graphs

This section analyses the training loss by graphing it at a high level.

The loss curve for all digits show visible inflection points (bumps), but is too high level to help understand the algorithm.

When this graph is decomposed into 'per digit' graphs, the interesting distinct 'per digit' curves appear, showing each digit is being refined semi-independently, with the model algorithm refining each digit separately.



In [ ]:
epochs_to_graph=1200

# Helper function to plot multiple lines
def lines(raw_lines_list, x=None, mode='lines', labels=None, xaxis='Epoch', yaxis='Loss', title = '', log_y=False, hover=None, all_epochs=True, **kwargs):

    lines_list = raw_lines_list if all_epochs==False else [row[:epochs_to_graph] for row in raw_lines_list]
    log_suffix = '' if log_y==False else ' (Log)'
    epoch_suffix = '' if all_epochs==False else ' (' + str(epochs_to_graph) + ' steps)'
    full_title = title + log_suffix + epoch_suffix

    if type(lines_list)==torch.Tensor:
        lines_list = [lines_list[i] for i in range(lines_list.shape[0])]
    if x is None:
        x=np.arange(len(lines_list[0]))
    if save_graph_to_file :
      fig = go.Figure(layout={})
      print(full_title)
    else:
      fig = go.Figure(layout={'title':full_title})

    fig.update_xaxes(title=xaxis)
    fig.update_yaxes(title=yaxis + log_suffix)
    for c, line in enumerate(lines_list):
        if type(line)==torch.Tensor:
            line = utils.to_numpy(line)
        if labels is not None:
            label = labels[c]
        else:
            label = c
        fig.add_trace(go.Scatter(x=x, y=line, mode=mode, name=label, hovertext=hover, **kwargs))
    if log_y:
        fig.update_layout(yaxis_type="log")
    if save_graph_to_file:
        fig.update_layout(margin=dict(l=10, r=10, t=10, b=10),width=1200,height=300)

    fig.show(bbox_inches="tight")

    if save_graph_to_file:
        filename = full_title.replace(" ", "").replace("(", "").replace(")", "").replace("&", "").replace(",", "").replace("%", "")   +'.pdf'
        pio.write_image(fig, filename)


if train_model:
  title_suffix = ' Loss Curves for ' + str(n_digits) + ' digit addition'
  per_token_losses = np.stack(per_token_train_losses_list, axis=0)

  line(train_losses_list,
      title=title_suffix)

  all_epochs = True;
  for i in range(2):
    lines([per_token_losses[:, i] for i in range(1+n_digits)]+[train_losses_list],
          labels = [f'digit {i}' for i in range(1+n_digits)]+['all_digits'],
          title='Per digit'+title_suffix,
          all_epochs=all_epochs)

    all_epochs = False;

  for i in range(1+n_digits):
    print('Final Loss for digit ' + str(i) + ' is ', per_token_losses[-1, i])

# Part 6: Training Loss Analysis - Single task (multiple digits)
The previous section graphed across all the (BA, UC1, US9) tasks. This section graphs one of the addition sub-tasks at a time:
- The all-digits graphs for one task (say BA) again show several inflection points but do not provide significant insights.
- The per digit graphs for one task (say BA) again show distinct per digit curves. More useful but no significant insights.


In [ ]:
# Graph per digit series using "normal" and "log" scale
def graph_perdigit(losslist, num_series, title_suffix, showlog, all_epochs=True):
    lines([losslist[i] for i in range(num_series)],
          labels = [f'digit {i}' for i in range(num_series)],
          title='PerDigit '+title_suffix,
          all_epochs=all_epochs)

    if showlog:
      lines([losslist[i] for i in range(num_series)],
            labels = [f'digit {i}' for i in range(num_series)],
            title='PerDigit '+title_suffix,
            all_epochs=all_epochs,
            log_y=True)

    if all_epochs==True :
      total_loss = 0
      for i in range(num_series):
        print('Final Loss for digit ' + str(i) + ' is', losslist[i][-1])
        total_loss += losslist[i][-1]
      print('Mean Loss is', total_loss/num_series)
      print()

## Base Add task graphs
Graphs token loss vs step in use case where only BA (not UC1 or US9) is needed to get the correct answer.

In [ ]:
if train_model:
  perc = (int)(100 * ba_total_cases / (ba_total_cases + uc1_total_cases + us9_total_cases))
  print('BA Loss' + ' (' + str(ba_total_cases) + ' cases, ' + str(perc) + '%)')

  the_title = 'BA Loss'

  # For use cases where use_carry1s is false for all columns simultaneously, so BA can be used on all digits
  line(ba_alldigits_loss, title='AllDigits '+the_title)

  # For each digit independently
  graph_perdigit(ba_perdigit_loss, n_digits, the_title, False, True)
  graph_perdigit(ba_perdigit_loss, n_digits, the_title, False, False)

## Use Carry 1 (excluding Use Sum 9) task graphs
Graphs token loss vs step where use_carry1s is used at least once over the digits columns (and Sum9 is not used at all)

In [ ]:
if train_model:
  perc = (int)(100 * uc1_total_cases / (ba_total_cases + uc1_total_cases + us9_total_cases))
  print( 'UC1 Loss (' + str(uc1_total_cases) + ' cases, ' + str(perc) + '%)' )
  the_title = 'UC1 Loss'

  lines([uc1_anydigits_loss],
        labels = ['at least 1 digit'],
        title='AllDigits '+the_title)

  # For each digit independently
  graph_perdigit(uc1_perdigit_loss, n_digits, the_title, False, True)
  graph_perdigit(uc1_perdigit_loss, n_digits, the_title, False, False)


## Use Sum 9 task graphs
Graphs token loss vs step where US9 is used at least once over the digits columns

In [ ]:
if train_model:
  perc = (int)(100 * us9_total_cases / (ba_total_cases + uc1_total_cases + us9_total_cases))
  print( 'US9 Loss (' + str(us9_total_cases) + ' cases, ' + str(perc) + '%)')
  the_title = 'US9 Loss'

  lines([us9_anydigits_loss],
        labels = ['any digits'],
        title='AllDigits '+the_title)

  # For each digit independently
  graph_perdigit(us9_perdigit_loss, n_digits, the_title, False, True)
  graph_perdigit(us9_perdigit_loss, n_digits, the_title, False, False)

# Part 7: Training Loss Analysis - Single digit (multiple tasks)
This section graphs show multiple tasks but only one digit. These graphs provide some insights, including:
- The lowest value digit (D0) has a simple, steep loss curve
- The middle value digits (D1, D2 and D3 in 5 digit addition) have similar loss curves
- The highest value digit (D4 in 5 digit addition) has a different loss curve from the middle digits, suggesting the alogrithm for this digit differs from the middle digits.  


## Per digit BA & UC1 task graphs
For each digit, graph the BasedAdd and UC1 tasks for curve comparison

In [ ]:
if train_model:
  for whichdigit in range(n_digits):

    the_title = 'Loss for BA & UC1 tasks for Digit ' + str(whichdigit)

    lines([ba_perdigit_loss[whichdigit]]+[uc1_perdigit_loss[whichdigit]],
          labels = ['BA']+['UC1'],
          title=the_title)
    lines([ba_perdigit_loss[whichdigit]]+[uc1_perdigit_loss[whichdigit]],
          labels = ['BA']+['UC1'],
          title=the_title,
          log_y=True)
    lines([ba_perdigit_loss[whichdigit]]+[uc1_perdigit_loss[whichdigit]],
          labels = ['BA']+['UC1'],
          title=the_title,
          all_epochs=False)

## Per digit BA, UC1 & US9 task graphs
For each digit, graph the BasedAdd, UC1 & US9 tasks for curve comparison.

The high variability (noise) in the US9 curve comes from the rareness of this use case. There are ~4 examples in each training batch of 64. A single prediction error adds significant loss. The average US9 loss matches the BA and UC1 curves.

In [ ]:
if train_model:
  for whichdigit in range(n_digits):

    the_title = 'Loss for BA, UC1 & US9 Tasks for Digit ' + str(whichdigit)

    lines([ba_perdigit_loss[whichdigit]]+[uc1_perdigit_loss[whichdigit]]+[us9_perdigit_loss[whichdigit]],
          labels = ['BA']+['UC1']+['US9'],
          title=the_title)
    lines([ba_perdigit_loss[whichdigit]]+[uc1_perdigit_loss[whichdigit]]+[us9_perdigit_loss[whichdigit]],
          labels = ['BA']+['UC1']+['US9'],
          title=the_title,
          log_y=True)
    lines([ba_perdigit_loss[whichdigit]]+[uc1_perdigit_loss[whichdigit]]+[us9_perdigit_loss[whichdigit]],
          labels = ['BA']+['UC1']+['US9'],
          title=the_title,
          all_epochs=False)

# Part 8A: Questions Set Up

Create sets of sample questions (by task) to ask the model to predict




In [ ]:
# Insert a number into the question
def insert_question_number(the_question, index, first_digit_index, the_digits, n):

  last_digit_index = first_digit_index + the_digits - 1

  for j in range(the_digits):
    the_question[index, last_digit_index-j] = n % 10
    n = n // 10


# Create a single question
def make_a_question(the_question, index, q1, q2):
  a = q1 + q2

  insert_question_number(the_question, index, 0, n_digits, q1)

  the_question[index, n_digits] = PLUS_INDEX

  insert_question_number( the_question, index, n_digits+1, n_digits, q2)

  if long_equals == True:
    the_question[index, 2*n_digits+1] = E_INDEX
    the_question[index, 2*n_digits+2] = Q_INDEX
    the_question[index, 2*n_digits+3] = U_INDEX
    the_question[index, 2*n_digits+4] = A_INDEX
    the_question[index, 2*n_digits+5] = L_INDEX
    the_question[index, 2*n_digits+6] = S_INDEX
    offset = 7
  else:
    the_question[index, 2*n_digits+1] = EQUALS_INDEX
    offset = 2

  insert_question_number(the_question, index, 2*n_digits + offset, n_digits+1, q1+q2)


# Create a batch of questions from a 2D matrix of ints
def make_questions(q_matrix):
  length = len(q_matrix)
  if length < 5:
    print("make_questions:Insufficent cases", length)

  questions = torch.zeros((length, n_ctx)).to(torch.int64)

  limit = 10 ** n_digits
  for i in range(length):
    if (q_matrix[i][0] < limit) and (q_matrix[i][1] < limit) :
      make_a_question(questions, i, q_matrix[i][0], q_matrix[i][1])

  return questions


def prediction_to_string(max_indices):
  answer = "".join([str(i) for i in utils.to_numpy(max_indices)[0]])
  return answer;

In [ ]:
# Analyse the question and return the use case as BA, MC, SimpleUS9 or CascadeUS9
def get_question_case(q):
  qa = utils.to_numpy(q)
  qn = qa[:2*n_digits+2]

  # Locate the MC and MS digits (if any)
  mc = torch.zeros( n_digits).to(torch.int64)
  ms = torch.zeros( n_digits).to(torch.int64)
  for dn in range(n_digits):
    if qn[dn] + qn[dn + n_digits + 1] == 9:
      ms[n_digits-1-dn] = 1
    if qn[dn] + qn[dn + n_digits +1] > 9:
      mc[n_digits-1-dn] = 1

  # Calculate the use case of a question
  if torch.sum(mc) == 0:
    return "BA"

  if torch.sum(ms) == 0:
    return "MC1"

  for dn in range(n_digits):
    if dn < n_digits-2 and mc[dn] == 1 and ms[dn+1] == 1 and ms[dn+2] == 1:
      return "CascadeUS9"

  for dn in range(n_digits):
    if dn < n_digits-1 and mc[dn] == 1 and ms[dn+1] == 1:
      return "SimpleUS9"

  return "MC1"

In [ ]:
# Manually create some questions that strongly test one use case


def make_ba_questions():
    return make_questions(
      [[12345, 33333],
      [33333, 12345],
      [45762, 33113],
      [888, 11111],
      [2362, 23123],
      [15, 81],
      [1000, 4440],
      [4440, 1000],
      [24033, 25133],
      [23533, 21133],
      [32500, 1],
      [31500, 1111],
      [5500, 12323],
      [4500, 2209],
      [ 33345, 66643], # =099988
      [ 66643, 33345], # =099988
      [10990, 44000],
      [60000, 30000],
      [10000, 20000]])


def make_uc1_questions():
    return make_questions(
      [[ 15, 45],
      [ 25, 55],
      [ 35, 59],
      [ 40035, 40049],
      [ 5025, 5059],
      [ 15, 65],
      [ 44000, 46000],
      [ 70000, 40000],
      [ 15000, 25000],
      [ 35000, 35000],
      [ 45000, 85000],
      [ 67000, 85000],
      [ 99000, 76000],
      [ 1500, 4500],
      [ 2500, 5500],
      [ 3500, 5900],
      [ 15020, 45091],
      [ 25002, 55019],
      [ 35002, 59019]])


def make_simple_us9_questions():
    return make_questions(
      [[ 55, 45],
      [ 45, 55],
      [ 45, 59],
      [ 35, 69],
      [ 25, 79],
      [ 15, 85],
      [ 15, 88],
      [ 15508, 14500],
      [ 14508, 15500],
      [ 24533, 25933],
      [ 23533, 26933],
      [ 32500, 7900],
      [ 31500, 8500],
      [ 550, 450],
      [ 450, 550],
      [ 10880, 41127],
      [ 41127, 10880],
      [ 12386, 82623]])


def make_cascade_us9_questions(clean = True):
    return make_questions(
      # These are two level UseSum9 cascades
      [[ 555, 445],
      [ 3340, 6660],
      [ 8880, 1120],
      [ 1120, 8880],
      [ 123, 877],
      [ 877, 123],
      [ 321, 679],
      [ 679, 321],
      [ 1283, 88786],
      # These are three level UseSum9 cascades
      [ 5555, 4445],
      [ 55550, 44450],
      [ 334, 666],
      [ 3340, 6660],
      [ 33400, 66600],
      [ 888, 112],
      [ 8880, 1120],
      [ 88800, 11200],
      [ 1234, 8766],
      [ 4321, 5679],
      # These are four level UseSum9 cascades
      [ 44445, 55555],
      [ 33334, 66666],
      [ 88888, 11112],
      [ 12345, 87655],
      [ 54321, 45679],
      [ 45545, 54455],
      [ 36634, 63366],
      [ 81818, 18182],
      [ 87345, 12655],
      [ 55379, 44621]])


# These questions focus mainly on 1 digit at a time
# (We're assuming that the 0 + 0 digit additions are trivial bigrams)
def make_step_questions():
    return make_questions(
      [[0, 1, 0],
      [1, 4, 3],
      [2, 5, 5],
      [3, 8, 1],
      [4, 40, 30],
      [5, 44, 46],
      [6, 400, 300],
      [7, 440, 460],
      [8, 800, 100],
      [9, 270, 470],
      [10, 600, 300],
      [11, 4000, 3000],
      [12, 4400, 4600],
      [13, 6000, 3000],
      [14, 7000, 4000],
      [15, 40000, 30000],
      [16, 44000, 46000],
      [17, 60000, 30000],
      [17, 70000, 40000],
      [19, 10000, 20000],
      [20, 15000, 25000],
      [21, 35000, 35000],
      [22, 45000, 85000],
      [23, 67000, 85000],
      [24, 99000, 76000],
      [25, 76000, 99000]])


# Returns 128 random and ~100 manually-chosen questions
def make_varied_questions():
  q0, _, _, _, _, _ = next(ds)
  q1 = make_ba_questions()
  q2 = make_uc1_questions()
  q3 = make_simple_us9_questions()
  q4 = make_cascade_us9_questions()
  q5 = make_step_questions()
  q6, _, _, _, _, _ = next(ds)

  questions = torch.vstack((q0.cuda(), q1.cuda(), q2.cuda(), q3.cuda(), q4.cuda(), q5.cuda(), q6.cuda()))

  return questions

In [ ]:
# Test that the get_question_case code works as expected
def unit_test_get_question_case_core(correct_case, questions):
  num_questions = questions.shape[0]
  print( correct_case, "#Questions=", num_questions)
  for i in range(num_questions):
    question_case = get_question_case(questions[i])
    if question_case != correct_case:
      print( "Case mismatch:", correct_case, question_case, questions[i])

def unit_test_get_question_case():
  unit_test_get_question_case_core( "BA", make_ba_questions())
  unit_test_get_question_case_core( "MC1", make_uc1_questions())
  unit_test_get_question_case_core( "SimpleUS9", make_simple_us9_questions())
  unit_test_get_question_case_core( "CascadeUS9", make_cascade_us9_questions())

unit_test_get_question_case()

In [ ]:
num_questions = 0;
correct_answers = 0;
verbose = True
total_mean_loss = 0.0


# Clear the question summary results
def clear_questions_results(title):
  global num_questions
  global correct_answers
  global verbose
  global total_mean_loss

  num_questions = 0
  correct_answers = 0
  total_mean_loss = 0

  if verbose:
    print(title)


# Ask model to predict answer for each question & collect results
def do_questions(questions):
    global num_questions
    global correct_answers
    global verbose
    global total_mean_loss

    num_questions = questions.shape[0]
    for question_num in range(num_questions):
      q = questions[question_num]

      # Run with no hook
      the_logits = model(q.cuda())

      q_2d = q.unsqueeze(0)
      losses_raw, max_indices = logits_to_tokens_loss(the_logits, q_2d.cuda())
      losses = loss_fn(losses_raw)
      mean_loss = utils.to_numpy(losses.mean())
      total_mean_loss = total_mean_loss + mean_loss

      model_answer_str = prediction_to_string(max_indices)
      model_answer_num = int(model_answer_str)

      i = n_digits*2 + 7 if long_equals == True else n_digits*2 + 2

      a = 0
      # 5 digit addition yields a 6 digit answer. Hence n_digits+1
      for j in range(n_digits+1):
        a = a * 10 + q[i+j]

      correct = (model_answer_num == a)
      if correct :
        correct_answers += 1

      if verbose:
        print(tokens_to_string(q), "ModelAnswer:", model_answer_str, "Correct:", correct, "Loss:", mean_loss )


# Print the question summary results
def print_questions_results(prefix, output_table):
  global num_questions
  global correct_answers
  global total_mean_loss

  output_table.add_row([prefix, num_questions, correct_answers, 100*correct_answers/num_questions, total_mean_loss/num_questions])

In [ ]:
# Build a test batch of 64 random and ~100 manually-chosen questions
varied_questions = make_varied_questions();

# Run the sample batch, gather the cache
model.reset_hooks()
model.set_use_attn_result(True)
sample_logits, sample_cache = model.run_with_cache(varied_questions.cuda())
print(sample_cache) # Gives names of datasets in the cache
sample_losses_raw, sample_max_indices = logits_to_tokens_loss(sample_logits, varied_questions.cuda())
sample_loss_mean = utils.to_numpy(loss_fn(sample_losses_raw).mean())
print("Sample Mean Loss", sample_loss_mean) # Loss < 0.04 is good

# attn.hook_z is a key "attention head output" hook key name
hook_l0_z_name = 'blocks.0.attn.hook_z'
hook_l1_z_name = 'blocks.1.attn.hook_z'
sample_attn_z = sample_cache[hook_l0_z_name]
print("Sample", hook_l0_z_name, sample_attn_z.shape) # gives [239, 18, 3, 170] = ???, n_ctx, n_heads, d_head
mean_attn_z = torch.mean(sample_attn_z, dim=0, keepdim=True)
print("Mean", hook_l0_z_name, mean_attn_z.shape) # gives [1, 18, 3, 170] = 1, n_ctx, n_heads, d_head

# hook_resid_post is the key "residual memory" hook point name
hook_l0_resid_post_name = 'blocks.0.hook_resid_post'
hook_l1_resid_post_name = 'blocks.1.hook_resid_post'
sample_resid_post = sample_cache[hook_l0_resid_post_name]
print("Sample", hook_l0_resid_post_name, sample_resid_post.shape) # gives [239, 18, 510] = ???, n_ctx, d_model
mean_resid_post = torch.mean(sample_resid_post, dim=0, keepdim=True)
print("Mean", hook_l0_resid_post_name, mean_resid_post.shape) # gives [1, 18, 510] = 1, n_ctx, d_model

# blocks.0.mlp.hook_post is the key "MLP layer" hook point name
hook_l0_mlp_hook_post_name = 'blocks.0.mlp.hook_post'
hook_l1_mlp_hook_post_name = 'blocks.1.mlp.hook_post'
sample_mlp_hook_post = sample_cache[hook_l0_mlp_hook_post_name]
print("Sample", hook_l0_mlp_hook_post_name, sample_mlp_hook_post.shape) # gives [239, 18, 2040] = ???, n_ctx, d_model*4
mean_mlp_hook_post = torch.mean(sample_mlp_hook_post, dim=0, keepdim=True)
print("Mean", hook_l0_mlp_hook_post_name, mean_mlp_hook_post.shape) # gives [1, 18, 2040] = 1, n_ctx, d_model*4

# Part 8B: Prediction Analysis - By Use Case
This section sets up BA, UC1 and US9 test cases that will be re-used in later experiments to show the impact of ablating heads or steps.



In [ ]:
global total_mean_loss
global num_questions

exp0_output = PrettyTable()
exp0_output.field_names = ["Case", "#Questions", "#Correct", "%Correct", "Mean loss"]
verbose = False

clear_questions_results("Simple BaseAdd cases")
do_questions(make_ba_questions())
print_questions_results("BaseAdd", exp0_output)
sum_total_mean_loss = total_mean_loss
sum_num_questions = num_questions

clear_questions_results("These are Use Carry 1 (UC1) examples (not UseSum9 examples)")
do_questions(make_uc1_questions())
print_questions_results("UseCarry1", exp0_output)
sum_total_mean_loss = sum_total_mean_loss + total_mean_loss
sum_num_questions = sum_num_questions + num_questions

clear_questions_results("These are simple (one level) UseSum9 exampless")
do_questions(make_simple_us9_questions())
print_questions_results("SimpleUS9", exp0_output)
sum_total_mean_loss = sum_total_mean_loss + total_mean_loss
sum_num_questions = sum_num_questions + num_questions

clear_questions_results("These are UseSum9 two, three and four level cascades")
do_questions(make_cascade_us9_questions())
print_questions_results("CascadeUS9", exp0_output)
sum_total_mean_loss = sum_total_mean_loss + total_mean_loss
sum_num_questions = sum_num_questions + num_questions

clear_questions_results("These questions focus on different steps")
do_questions(make_step_questions())
print_questions_results("Steps", exp0_output)
sum_total_mean_loss = sum_total_mean_loss + total_mean_loss
sum_num_questions = sum_num_questions + num_questions

exp0_output.add_row(["OVERALL", sum_num_questions, "", "", sum_total_mean_loss])

print_config()
print()
print(exp0_output.get_formatted_string(out_format=table_out_format))

# Part 9A: Prediction Analysis - Model Shape

The "Prediction" sections analyses the model after it has been trained, by looking at how it predicts answers to questions. This section shows the shape of the data available. No insights derived.


Get some new tokens from the data generator

In [ ]:
tokens, base_adds, make_carry1s, sum9s, use_carry1s, use_sum9s = next(ds)

print("tokens.shape", tokens.shape)
print("sample tokens", tokens[:4])
print(tokens_to_string(tokens[0]))
print(tokens_to_string(tokens[1]))
print(tokens_to_string(tokens[2]))
print(tokens_to_string(tokens[3]))

Run the model on the tokens

In [ ]:
original_logits, cache = model.run_with_cache(tokens)
print("original_logits.numel", original_logits.numel())

Get key weight matrices:

In [ ]:
W_E = model.embed.W_E[:-1]
print("W_E shape:", W_E.shape)

W_neur = W_E @ model.blocks[0].attn.W_V @ model.blocks[0].attn.W_O @ model.blocks[0].mlp.W_in
print("W_neur shape:", W_neur.shape)

W_logit = model.blocks[0].mlp.W_out @ model.unembed.W_U
print("W_logit shape:", W_logit.shape)

In [ ]:
per_token_train_losses_original, _ = logits_to_tokens_loss(logits, tokens)
original_loss = loss_fn(per_token_train_losses_original).mean()
print("Original Loss:", utils.to_numpy(original_loss))

In [ ]:
pattern_a = cache["pattern", 0, "attn"][:, :, -1, 0]
pattern_b = cache["pattern", 0, "attn"][:, :, -1, 1]
neuron_acts = cache["post", 0, "mlp"][:, -1, :]
neuron_pre_acts = cache["pre", 0, "mlp"][:, -1, :]

for param_name, param in cache.items():
    print(param_name + ' shape:', param.shape)

# Part 9B: Prediction Analysis - Attention Patterns
Attention patterns show which token(s) the model's attention heads are paying attention to in each step of the prediction calculation.

For the default CoLab set up, the  model has 3 attention heads, and performs 5 digit addition. The attention pattern is 18 by 18 squares (as 54321+77779=132100 is 18 tokens). Time proceeds vertically downwards, with one additional token being revealed horizontally at each step, giving the overall triangle shape. This visualisation provided insights. After the question is fully revealed (at step 11), each head starts attending to pairs of question digits from left to right (i.e. high-value digits before lower-value digits) giving the “double staircase" shape. The three heads attend to a given digit pair in three different steps, giving a time ordering of heads.

### Show attention patterns for some randomly chosen tokens

In [ ]:
def show_token_attention_patterns(index, layer, token_at_index, use_case):

  the_tokens = [str(token) for token in token_at_index.tolist()]
  if layer == 0:
    tokens_str = tokens_to_string(the_tokens)
    print("Attention patterns for", tokens_str)

  attention_pattern=cache["pattern", layer, "attn"][index]
  display(cv.attention.attention_patterns(
      tokens=the_tokens,
      attention=attention_pattern,
      attention_head_names=[f"L{layer}H{i}" for i in range(n_heads)],
  ))


sample_size = 3

# Show attention patterns for some randomly chosen tokens
for i in range(sample_size):
  for layer in range(n_layers):
    show_token_attention_patterns(i, layer, tokens[i], "Misc")


In [ ]:
if save_graph_to_file:

  tokens_str = []
  for i in range(n_heads):
    one_token_str = []
    for j in tokens[i]:
      one_token_str.append(str(utils.to_numpy(j)))
    tokens_str.append(one_token_str)

  # Refer https://github.com/callummcdougall/CircuitsVis/blob/main/python/circuitsvis/circuitsvis_demo.ipynb

  html_object = cv.attention.from_cache(
      cache = cache,
      tokens = tokens_str, # list of list of strings
      return_mode = "html",
  )

  # Create a CoLab file containing the attention pattern(s) in HTML
  filename = "AttentionPattern" + str(n_digits) + "Digits" + str(n_heads) + "Heads.html"
  with open(filename, "w") as f:
      f.write(html_object.data)

  # Manually download the CoLab "html" file and open in your local browser.
  # Install and use the Edge extension "FireShot" to save a portion of the HTML page as a PDF

### Show attention patterns for some tokens which BA only



In [ ]:
any_use_carry1s = torch.any(use_carry1s.bool(), dim=1)
no_use_carry1s = ~ any_use_carry1s
ba_num_cases = utils.to_numpy(torch.sum(no_use_carry1s))
if ba_num_cases >= sample_size :
  print(f"Attention patterns for first few BA-only tokens ({ba_num_cases} of {tokens.shape[0]})")
  ba_tokens = tokens[no_use_carry1s==1]
  for i in range(sample_size):
    for layer in range(n_layers):
      show_token_attention_patterns(i, layer, ba_tokens[i], "BAOnly")

Show attention patterns for some tokens which UC1 (and not US9)

In [ ]:
num_use_carry1s = torch.sum(use_carry1s, dim=1)
any_use_carry1s = torch.where( num_use_carry1s != 0, 1, 0 ) # At least one digit uses UC1
num_sum9s = torch.sum(use_sum9s, dim=1)
no_sum9s = torch.where( num_sum9s == 0, 1, 0 ) # No digits have Sum9 true
filtered_cases = any_use_carry1s & no_sum9s
uc1_num_cases = utils.to_numpy(torch.sum(filtered_cases))
if uc1_num_cases >= sample_size :
  print(f"Attention patterns for first few UC1-only (and not US9) tokens ({uc1_num_cases} of {tokens.shape[0]})")
  uc1_tokens = tokens[filtered_cases==1]
  for i in range(sample_size):
    for layer in range(n_layers):
      show_token_attention_patterns(i, layer, uc1_tokens[i], "UC1")

Show attention patterns for some tokens which US9

In [ ]:
num_sum9s = torch.sum(use_sum9s, dim=1)
any_sum9s = torch.where( num_sum9s != 0, 1, 0 ) # At least one digit uses Sum9
us9_cases = utils.to_numpy(torch.sum(any_sum9s))
if us9_cases >= sample_size :
  print(f"Attention patterns for first few US9 tokens ({us9_cases} of {tokens.shape[0]})")
  us9_tokens = tokens[any_sum9s==1]
  for i in range(sample_size):
    for layer in range(n_layers):
      show_token_attention_patterns(i, layer, us9_tokens[i], "US9")

# Part 10A: Set Up Run Framework

Create way to get model to predict sample question answers and analysis/show results

In [ ]:
exp1_case_counts = {}


def count_question_cases(questions):
  global exp1_case_counts

  exp1_case_counts = {}

  for i in range(questions.shape[0]):
    q_case = get_question_case(questions[i])

    if q_case in exp1_case_counts:
      # If the key is already in the dictionary, increment its count
      exp1_case_counts[q_case] += 1
    else:
      # If the key is not in the dictionary, add it with a count of 1
      exp1_case_counts[q_case] = 1

In [ ]:
# Compare each digit in the answer. Returns a yyNNy pattern where y means the digits match and N means a failure
def get_digit_accuracy_pattern(a_int, answer_str):
  a_str = str(a_int.cpu().numpy()).zfill(n_digits+1)
  match_str = ""
  for i in range(n_digits+1):
      if answer_str[i] == a_str[i]:
          match_str += "y"  # Matching digit
      else:
          match_str += "N"  # Non-matching digit
  return match_str

In [ ]:
# Build up a list of success/failure patterns found, and the frequency of each pattern
exp1_pattern_fails = {}


def clear_pattern_fails():
  global exp1_pattern_fails

  exp1_pattern_fails = {}


def add_pattern_fail(step_key):
  global exp1_pattern_fails

  if step_key in exp1_pattern_fails:
    # If the key is already in the dictionary, increment its count
    exp1_pattern_fails[step_key] += 1
  else:
    # If the key is not in the dictionary, add it with a count of 1
    exp1_pattern_fails[step_key] = 1


def get_pattern_fails():
  global exp1_pattern_fails

  results = ""
  if len(exp1_pattern_fails) > 0 :
    sorted_fails = dict(sorted(exp1_pattern_fails.items(), key=lambda item: item[1], reverse=True))
    for key, value in sorted_fails.items():
      results = results + key + "=" + str(value) + ", "

  return results


def get_pattern_fails_total():
  global exp1_pattern_fails

  if len(exp1_pattern_fails) == 0:
    return 0

  total_sum = 0
  for key, value in exp1_pattern_fails.items():
      if isinstance(value, int):
          total_sum += value
  return total_sum

In [ ]:
# Build up a count of failure cases
exp1_case_fails = {}


def clear_case_fails():
  global exp1_case_fails

  exp1_case_fails = {}


def add_case_fail(case_key):
  global exp1_case_fails

  if case_key in exp1_case_fails:
    # If the key is already in the dictionary, increment its count
    exp1_case_fails[case_key] += 1
  else:
    # If the key is not in the dictionary, add it with a count of 1
    exp1_case_fails[case_key] = 1


def total_case_fails():
  global exp1_case_fails

  answer = 0
  for _, value in exp1_case_fails.items():
    answer = answer + value
  return answer


def get_case_fails():
  global exp1_case_fails
  global exp1_case_counts

  results = ""
  num_results = len(exp1_case_fails)
  if num_results > 0:
    sorted_fails = dict(sorted(exp1_case_fails.items(), key=lambda item: item[1], reverse=True))

    for key, value in sorted_fails.items():
      percent = round(100 * value / exp1_case_counts[key])
      results = results + "%" + key + "=" + str(percent)+ ", "

  return results

In [ ]:
def predict_experiment_question(questions, the_hook, the_threshold):

  exp1_loss_mean = 0

  clear_case_fails()
  clear_pattern_fails()
  count_question_cases(questions)

  num_questions = questions.shape[0]
  for question_num in range(num_questions):
    q = questions[question_num]

    model.reset_hooks()
    exp_logits = model.run_with_hooks(q.cuda(), return_type="logits", fwd_hooks=the_hook)

    q_2d = q.unsqueeze(0)
    exp_losses_raw, exp_max_indices = logits_to_tokens_loss(exp_logits, q_2d.cuda())
    exp1_loss_mean = utils.to_numpy(loss_fn(exp_losses_raw).mean())
    exp_answer_str = prediction_to_string(exp_max_indices)

    i = 17 if long_equals == True else 12
    a = q[i+0] * 100000 + q[i+1] * 10000 + q[i+2] * 1000 + q[i+3] * 100 + q[i+4] * 10 + q[i+5] * 1;

    # Only show the question if the loss exceeds the threshold (because of the ablated step)
    if exp1_loss_mean > the_threshold:
      match_str = get_digit_accuracy_pattern( a, exp_answer_str )
      # Only count the question if the model got the question wrong
      if 'N' in match_str:
        the_case = get_question_case(q)
        add_case_fail(the_case)
        add_pattern_fail(match_str)
        if verbose:
          print(tokens_to_string(q), "ModelAnswer:", exp_answer_str, "Matches:", match_str, "Loss:", exp1_loss_mean, "Case:", the_case )

  return exp1_loss_mean

# Part 10B: Ablate ALL Heads in EACH step. What is the impact on Loss?

Here we ablate all heads in each step and see if loss increases to show which steps are used by the algorithm. Unused steps can be excluded from further analysis.

This section overrides (ablates) the model memory (residual stream) at each step. It confirms that for:
- n_digits = 5, n_layers = 1 : the addition algorithm does **not** use any data generated in steps 0 to 10 inclusive. In these steps the model has **not** yet seen the full question and every digit in the question is independent of every other digit, making accurate answer prediction infeasible. The model also does not use the last (17th) step. Therefore, the addition is started and completed in 6 steps (11 to 16)
- n_digits = 5, n_layers = 2 : the addition algorithm does **not** use any data generated in steps 0 to 7 inclusive. The model also does not use the last (17th) step. Therefore, the addition is started and completed in 9 steps (8 to 16).

In [ ]:
# Experiment 1 - Ablate all heads in each step to see what question digits and tasks then fail.

exp1_step = 0  # zero-based step to ablate
exp1_threshold = 0.01
exp1_questions = varied_questions
exp1_output = PrettyTable()
exp1_output.field_names = ["Step", "Important?", "% Fails", "% Fails by Case", "# Fails by Patterns"]

min_useful_step = -1 # Minimum useful step where loss increases on ablation
max_useful_step = -1 # Maximum useful step where loss increases on ablation
verbose = False


def exp1_hook(value, hook):
  #print( "In hook", hook_l0_resid_post_name, exp1_ablate, exp1_step, value.shape) # Get [64, 18, 510] = batch_size, num_tokens, d_model

  # Copy the mean resid post values in step N to all the batch questions
  value[:,exp1_step,:] = mean_resid_post[0,exp1_step,:].clone()
  #value[:,exp1_step,:] = 0


if n_digits >= 5 :
  exp1_fwd_hooks = [(hook_l0_resid_post_name, exp1_hook)]

  num_questions = exp1_questions.shape[0]

  for exp1_step in range(n_ctx):
    loss_mean = predict_experiment_question(exp1_questions, exp1_fwd_hooks, exp1_threshold)

    num_fails = total_case_fails()
    perc_fails = 0
    description = "No" if num_fails == 0 else "Yes"
    if num_fails > 0:
      perc_fails = round(100 * num_fails / num_questions)

      if min_useful_step == -1:
        min_useful_step = exp1_step
      max_useful_step = exp1_step

    exp1_output.add_row([exp1_step, description, perc_fails, get_case_fails(), get_pattern_fails()])


  print_config()
  print("num_questions=", num_questions, "min_useful_step=", min_useful_step, "max_useful_step=", max_useful_step )
  print()
  print(exp1_output.get_formatted_string(out_format=table_out_format))

# Part 10C: Ablate the MLP in EACH step. What is the impact on overall Loss?
Ablating the MLP in each layer in each step and seeing if the loss increases shows which head+layer+MLP are used by the algorithm.


In [ ]:
exp3_useful_cells = [[0 for _ in range(n_ctx)] for _ in range((n_heads + 1) * n_layers)]

In [ ]:
def add_useful_head( the_step, the_layer, the_head, perc_fails ):
  global exp3_useful_cells

  the_row = the_layer * (n_heads+1) + the_head

  if exp3_useful_cells[the_row][the_step] == 0 :
    exp3_useful_cells[the_row][the_step] = perc_fails
  else:
    print( "add_useful_head: Bad index", the_row, the_step)

  exp3_useful_cells[the_row][the_step] = perc_fails


def add_useful_mlp( the_step, the_layer, perc_fails ):
  add_useful_head( the_step, the_layer, n_heads, perc_fails )


def print_cell_line( prefix, cell_list ):
  answer = prefix
  for value in cell_list:
    if value > 99:
      value = 99

    if value == 0:
      answer = answer + "   "
    else:
      answer = answer + " " + str(value).zfill(2)
  print(answer)


def print_useful_cells():
  global exp3_useful_cells

  steps = [i for i in range(n_ctx)]

  print_cell_line("Step", steps)
  print()
  for i in range((n_heads + 1) * n_layers):
    head = i % (n_heads + 1)
    layer = i // (n_heads + 1)
    title = ( "L" + str(layer) + "H" + str(head) ) if head < n_heads else "MLP "
    print_cell_line(title, exp3_useful_cells[i])

In [ ]:
exp4_step = 0 # zero-based step to ablate. 0 to say 17
exp4_layer = 0 # zero-based layer to ablate. 0 to 1
exp4_threshold = 0.12
exp4_questions = varied_questions
exp4_output = PrettyTable()
exp4_output.field_names = ["Step", "MLP Layer", "% Fails", "% Fails by Case", "# Fails by Patterns"]


def exp4_hook(value, hook):
  global exp4_step

  #print( "In hook", hook_l0_mlp_hook_post_name, value.shape) # Get [1, 18, 2040] = ???, n_ctx, ???

  # Copy the mean resid post values in step N to the MLP
  value[:,exp4_step,:] =  mean_mlp_hook_post[:,exp4_step,:].clone() # Mean ablate


def exp4_perform_core():
  global exp4_step
  global exp4_threshold
  global exp4_questions
  global exp4_output

  the_hook = [(hook_l0_mlp_hook_post_name, exp4_hook)] if exp4_layer == 0 else [(hook_l1_mlp_hook_post_name, exp4_hook)]
  loss_mean = predict_experiment_question(exp4_questions, the_hook, exp4_threshold)

  num_fails = total_case_fails()
  if num_fails > 0:
    perc_fails = round(100 * num_fails / exp4_questions.shape[0])
    exp4_output.add_row([exp4_step, exp4_layer, perc_fails, get_case_fails(), get_pattern_fails()])

    add_useful_mlp( exp4_step, exp4_layer, perc_fails )


def exp4_perform(title):
  global exp4_step
  global exp4_layer
  global exp4_questions
  global exp4_output

  if n_digits >= 5 :
    print_config()
    print(title, exp4_questions.shape[0])
    print()

    for exp4_step in range(n_ctx):
      for exp4_layer in range(n_layers):
        exp4_perform_core()

    print(exp4_output.get_formatted_string(out_format=table_out_format))


verbose = False
exp4_questions = varied_questions
exp4_perform("# varied questions=")

# Part 10D: Ablate EACH head in EACH step. What is the impact on Digit & Task Loss?
Ablating each head in each layer in each step and seeing if the loss increases shows which head+layer+step are used by the algorithm.

In 2 layer model, most second layer head+step are not used.
The results are diagrammed in StaircaseA3L2H3_Part1.drawio

In [ ]:
exp7_step = 0 # zero-based step to ablate. 0 to say 17
exp7_layer = 0 # zero-based layer to ablate. 0 to 1
exp7_head = 0 # zero-based head to ablate. 0 to 2
exp7_threshold = 0.12
exp7_questions = varied_questions
exp7_output = PrettyTable()


def exp7_reset(title):
  global exp7_questions
  global exp7_output

  exp7_output = PrettyTable()
  exp7_output.field_names = ["Step", "Layer", "Head", "% Fails", "% Fails by Case", "# Fails by Patterns"]

  print_config()
  print(title, exp7_questions.shape[0])
  print()


def exp7_hook(value, hook):
  global exp7_step
  global exp7_head

  # print( "In hook", hook_l0_z_name, value.shape) # Get [1, 18, 3, 170] = ???, n_ctx, n_heads, d_head

  # Copy the mean resid post values in step N to all the batch questions
  value[:,exp7_step,exp7_head,:] = mean_attn_z[:,exp7_step,exp7_head,:].clone() # Mean ablate


def exp7_perform_core():
  global exp7_step
  global exp7_head
  global exp7_threshold
  global exp7_questions

  the_hook = [(hook_l0_z_name, exp7_hook)] if exp7_layer == 0 else [(hook_l1_z_name, exp7_hook)]
  loss_mean = predict_experiment_question(exp7_questions, the_hook, exp7_threshold)

  num_fails = total_case_fails()
  if num_fails > 0:
    perc_fails = round(100 * num_fails / exp7_questions.shape[0])
    exp7_output.add_row([exp7_step, exp7_layer, exp7_head, perc_fails, get_case_fails(), get_pattern_fails()])

    add_useful_head( exp7_step, exp7_layer, exp7_head, perc_fails )

def exp7_perform(title):
  global exp7_step
  global exp7_layer
  global exp7_head

  if n_digits >= 5 :
    exp7_reset(title)
    for exp7_step in range(min_useful_step, max_useful_step+1):
      for exp7_layer in range(n_layers):
        for exp7_head in range(n_heads):
          exp7_perform_core()


def exp7_print_results():
  global exp7_output

  print(exp7_output.get_formatted_string(out_format=table_out_format))

In [ ]:
verbose = False
exp7_questions = varied_questions
exp7_perform("# varied questions=")
exp7_print_results()

# Part 11A - BA Analysis
Just processing BA questions (with n_digits = 5, n_layers = 2, n_heads = 3):
- S0 to S11 and S17 are not relevant
- L1 is not relevant
- L0H1 is not relevant

In [ ]:
verbose = False
exp7_questions = make_ba_questions()
exp7_perform("# BA questions=")
exp7_print_results()

# Part 11B - UC1 Analysis
Just processing UC1 questions (with n_digits = 5, n_layers = 2, n_heads = 3):
- S0 to S10 and S17 are not relevant
- L1 is not relevant


In [ ]:
verbose = False
exp7_questions = make_uc1_questions()
exp7_perform("# UC1 questions=")
exp7_print_results()

# Part 11C - Simple US9 Analysis
Just processing SimpleUS9 questions (with n_digits = 5, n_layers = 2, n_heads = 3):
- S0 to S7 and S17 are not relevant
- L1 is not relevant


In [ ]:
verbose = False
exp7_questions = make_simple_us9_questions()
exp7_perform("# SimpleUS9 questions")
exp7_print_results()

# Part 11D - Cascade US9 Analysis
Just processing CascadeUS9 questions (with n_digits = 5, n_layers = 2, n_heads = 3):
- S0 to S7 and S17 are not relevant


In [ ]:
verbose = False
exp7_questions = make_cascade_us9_questions()
exp7_perform("# CascadeUS9 questions")
exp7_print_results()

# Part 12A: Show the cells that do useful calculations

Show the percentage failure rate (incorrect prediction) when individual heads and MLPs are ablated.

A 1% failure rate may be zeroed through longer training.

In [ ]:
print_config()
print()
print_useful_cells()

# Part 12B: Shape of calculation

Combing the Attention Pattern "focus" information (Part 9B), the "unneeded Step+Heads" information (Part 10B), the "Unneeded Step+Layer+MLP" information (Part10C) we can get an initial picture of what model parts meaningfully contribute to the model calculations.

StaircaseA3L2H3_Part1.svg shows this information for n_layers=2, n_heads=3. Steps 0..7 and 11 are excluded as unneeded. Cells containing "X" are unneeded.

StaircaseA3L2H3_Part2.svg

# Part 13A: Test Hypothesis D4.T1 is calculated at S11.L0.H2

When n_digits = 5, n_layers = 2 and n_heads = 3:

- We claim D4.T1 is calculated at S11.L0.H2. When this cell is ablated we expect A4 and A5 calcuations to be less accurate.

In [ ]:
if n_digits == 5 and n_layers == 2 and n_heads == 3:
  verbose = True
  exp7_step = 11
  exp7_layer = 0
  exp7_head = 2
  exp7_threshold = 0.12
  exp7_questions = make_questions(
    [[15508, 14500],
    [14508, 15500],
    [24533, 25933],
    [23533, 26933],
    [10880, 41127],
    [41127, 10880],
    [12386, 82623],
    [41127, 80880],
    [52386, 82623],
    [41127, 70880],
    [32386, 82623]])

  def null_hook(value, hook):
    verbose = True

  loss_mean = predict_experiment_question(exp7_questions, null_hook, exp7_threshold)
  print( "No ablation mean loss", loss_mean )

  exp7_reset("# D4.T1 questions=")
  exp7_perform_core()
  exp7_print_results()